In [5]:
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
xgb.set_config(verbosity=0)

데이터 셋 준비

In [6]:
# 심장질환 데이터셋
!wget -q https://raw.githubusercontent.com/rickiepark/handson-gb/main/Chapter06/heart_disease.csv

In [7]:
import pandas as pd
df =pd.read_csv('/content/heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


모델 : XGBClassifier

In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [10]:
# 데이터 나누기 X,y
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [11]:
# 분류모델중에 성능이 좀 괜찮은 알고리즘 logist
model = XGBClassifier(booster = 'gbtree',objective = 'binary:logistic')

In [14]:
# 교차검증
from sklearn.model_selection import cross_val_score
import numpy as np
scores = cross_val_score(model,X,y, cv=5)
# 정확도 측정
print(f"정확도:{np.round(scores,2)}")
# 정확도 평균 측정
print(f"정확도 평균:{scores.mean()}")

정확도:[0.82 0.8  0.77 0.78 0.77]
정확도 평균:0.7886885245901639


StratifiedKFold
  - kfold 한쪽으로 몰리는 것 을 방지하기위해서 target 의 개수를 동일하게 가져간다

In [15]:
from sklearn.model_selection import StratifiedKFold
kfold =  StratifiedKFold(n_splits=5, shuffle=True,random_state=2)

In [20]:
np.unique(y,return_counts=True),  np.bincount(y)

((array([0, 1]), array([138, 165])), array([138, 165]))

In [23]:
print(f"전체 데이터의 클래스 분포 : {np.bincount(y)}")
print()
for plit_no , (train_idx,test_idx) in enumerate(kfold.split(X,y)):
  print(f"{plit_no+1}번째 훈련 폴드 : {np.bincount(y[train_idx])}")
  print(f"{plit_no+1}번째 검증 폴드 : {np.bincount(y[test_idx])}")
  print()

전체 데이터의 클래스 분포 : [138 165]

1번째 훈련 폴드 : [110 132]
1번째 검증 폴드 : [28 33]

2번째 훈련 폴드 : [110 132]
2번째 검증 폴드 : [28 33]

3번째 훈련 폴드 : [110 132]
3번째 검증 폴드 : [28 33]

4번째 훈련 폴드 : [111 132]
4번째 검증 폴드 : [27 33]

5번째 훈련 폴드 : [111 132]
5번째 검증 폴드 : [27 33]



GridSearchCV 와 RandomizedSearchCV

In [28]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)

In [30]:
def grid_search(params, random=False):
  xgb = XGBClassifier(booster = 'gbtree',objective = 'binary:logistic',random_state=2,verbosity=0,use_label_encoder=False)
  kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)
  if random:
    grid = RandomizedSearchCV(xgb,params,cv=kfold,n_iter=20,n_jobs=-1,random_state=2)
  else:
    grid = GridSearchCV(xgb,params,cv=kfold,n_jobs=-1)

  grid.fit(X,y)
  best_params = grid.best_params_
  print(f"best params:{best_params}")
  best_score = grid.best_score_
  print(f"best score:{best_score}")

XGBoost 하이퍼 파라메터 튜닝

In [31]:
grid_search(params ={ 'n_estimators':[100,200,400,800]})

best params:{'n_estimators': 400}
best score:0.7957377049180329


In [32]:
grid_search(params ={ 'learning_date':[0.01,0.05,0.1,0.2,0.3,0.4,0.5]})

best params:{'learning_date': 0.01}
best score:0.7925136612021858


In [33]:
grid_search(params ={ 'max_depth':[2,3,5,6,8]})

best params:{'max_depth': 6}
best score:0.7925136612021858


In [ ]:
# gamma : : 트리가 추가로 분할되기 위한 최소 손실 감소량 클수록 보수적인 분할

In [34]:
grid_search(params ={ 'gamma':[0,0.01,0.1,0.5,1.2]})

best params:{'gamma': 1.2}
best score:0.8022404371584699


In [36]:
grid_search(params ={ 'min_child_weight':[1,2,3,4,5]})

best params:{'min_child_weight': 3}
best score:0.7989617486338798


In [37]:
grid_search(params ={ 'subsample':[0.5,0.7,0.8,0.9,1]})

best params:{'subsample': 0.5}
best score:0.7956284153005464


In [38]:
grid_search(params ={ 'colsample_bytree':[0.5,0.7,0.8,0.9,1]})

best params:{'colsample_bytree': 0.5}
best score:0.81224043715847


조기종료
  - eval_set과 eval_metric

In [40]:
# eval_set과
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =  train_test_split(X,y,random_state=2)
model = XGBClassifier(booster = 'gbtree',objective = 'binary:logistic')
eval_set = [(X_test,y_test)]
eval_metric = 'error'
model.fit(X_train,y_train,eval_metric = eval_metric,eval_set=eval_set)
y_pred = model.predict(X_test)
# 평가
accuracy = accuracy_score(y_test,y_pred)
print(f"정확도:{accuracy}")

[0]	validation_0-error:0.27632
[1]	validation_0-error:0.15789
[2]	validation_0-error:0.07895
[3]	validation_0-error:0.09211
[4]	validation_0-error:0.10526
[5]	validation_0-error:0.10526
[6]	validation_0-error:0.10526
[7]	validation_0-error:0.13158
[8]	validation_0-error:0.13158
[9]	validation_0-error:0.13158
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.14474
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.15789
[14]	validation_0-error:0.15789
[15]	validation_0-error:0.15789
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.15789
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.17105
[23]	validation_0-error:0.15789
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.17105
[28]	validation_0-error:0.17105
[29]	validation_0-error:0.17105
[30]	validation_0-error:0.17105
[31]	validation_0-

early_stopping_rounds : 이 값기준으로 변화가 없으면 스탑

In [42]:
model = XGBClassifier(booster = 'gbtree',objective = 'binary:logistic')
eval_set = [(X_test,y_test)]
eval_metric = 'error'
model.fit(X_train,y_train,eval_metric = eval_metric,eval_set=eval_set,early_stopping_rounds = 10, verbose=True)
y_pred = model.predict(X_test)
# 평가
accuracy = accuracy_score(y_test,y_pred)
print(f"정확도:{accuracy}")

[0]	validation_0-error:0.27632
[1]	validation_0-error:0.15789
[2]	validation_0-error:0.07895
[3]	validation_0-error:0.09211
[4]	validation_0-error:0.10526
[5]	validation_0-error:0.10526
[6]	validation_0-error:0.10526
[7]	validation_0-error:0.13158
[8]	validation_0-error:0.13158
[9]	validation_0-error:0.13158
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.14474
[12]	validation_0-error:0.15789
정확도:0.9210526315789473


In [43]:
model = XGBClassifier(booster = 'gbtree',objective = 'binary:logistic',n_estimators = 5000)
eval_set = [(X_test,y_test)]
eval_metric = 'error'
model.fit(X_train,y_train,eval_metric = eval_metric,eval_set=eval_set,early_stopping_rounds = 100, verbose=True)
y_pred = model.predict(X_test)
# 평가
accuracy = accuracy_score(y_test,y_pred)
print(f"정확도:{accuracy}")

[0]	validation_0-error:0.27632
[1]	validation_0-error:0.15789
[2]	validation_0-error:0.07895
[3]	validation_0-error:0.09211
[4]	validation_0-error:0.10526
[5]	validation_0-error:0.10526
[6]	validation_0-error:0.10526
[7]	validation_0-error:0.13158
[8]	validation_0-error:0.13158
[9]	validation_0-error:0.13158
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.14474
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.15789
[14]	validation_0-error:0.15789
[15]	validation_0-error:0.15789
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.15789
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.17105
[23]	validation_0-error:0.15789
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.17105
[28]	validation_0-error:0.17105
[29]	validation_0-error:0.17105
[30]	validation_0-error:0.17105
[31]	validation_0-

하이퍼 파라메터 결합
  - 하나씩 하이퍼파라메터 조사하기

In [44]:
grid_search(params = {'n_estimators':[2,25,50,75,100]})

best params:{'n_estimators': 100}
best score:0.7925136612021858


In [45]:
grid_search(params = {
    'n_estimators':[2,25,50,75,100],
    'max_depth':[1,2,3,4,6,7,8]
                      })

best params:{'max_depth': 1, 'n_estimators': 50}
best score:0.838688524590164


In [46]:
grid_search(params = {
    'n_estimators':[2,25,50,75,100],
    'max_depth':[1,2,3,4,6,7,8],
    'learning_rate':[0.01,0.05,0.1,0.2,0.3,0.4,0.5]
                      })

best params:{'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50}
best score:0.838688524590164
